In [1]:
import tensorflow as tf

In [5]:
import glob
import zipfile

from sklearn.utils import shuffle
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend

In [2]:
!pwd

/Users/vidyadharbendre/nn_workspace/learn_neural_network_using_examples/jupyter_notebooks/RNN


In [3]:
!ls -la

total 3728
drwxr-xr-x   7 vidyadharbendre  staff     224 Apr 26 14:48 .
drwxr-xr-x  44 vidyadharbendre  staff    1408 Apr 25 22:07 ..
drwxr-xr-x@  8 vidyadharbendre  staff     256 Apr 26 14:40 .ipynb_checkpoints
-rw-r--r--@  1 vidyadharbendre  staff  467265 Apr 25 22:07 DNN_1.ipynb
-rw-r--r--@  1 vidyadharbendre  staff  713247 Apr 26 14:38 LSTM_3.ipynb
-rw-r--r--@  1 vidyadharbendre  staff  713513 Apr 25 22:07 RNN_2.ipynb
-rw-r--r--@  1 vidyadharbendre  staff    1762 Apr 26 14:48 Tagore_4.ipynb


In [6]:
filelist = glob.glob("../../data/Tagore/data/*.txt")
len(filelist)

20

In [8]:
text_data = []
for file in filelist:
    with open(file, "r") as file:
        text_data.append(file.read())

In [14]:
len(text_data)

20

In [13]:
text_data[0]

'The Project Gutenberg EBook of Stray Birds, by Rabindranath Tagore\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Stray Birds\n\nAuthor: Rabindranath Tagore\n\nPosting Date: March 27, 2010 [EBook #6524]\nRelease Date: September, 2004\nFirst Posted: December 25, 2002\n\nLanguage: English\n\nCharacter set encoding: ASCII\n\n*** START OF THIS PROJECT GUTENBERG EBOOK STRAY BIRDS ***\n\n\n\n\nProduced by Chetan K. Jain and Eric Eldred\n\n\n\n\n\n\n\n\n\nStray Birds\n\n\nBy Rabindranath Tagore\n\n\n[translated from Bengali to English by the author]\n\n\nNew York: The Macmillan Company, 1916\n\n\n[Frontispiece in color by Willy Pogany]\n\n\n\nTo\nT. HARA\nof\nYokohama\n\n\n\n\n1\nStray birds of summer come to my window to sing and fly away.\nAnd yellow leaves of autumn, which

In [15]:
text_data = []
for file in filelist:
    with open(file, "r") as file:
        file_content = file.read()
        text_data.append(file_content)
#        print(len(file_content.split(' ')))

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

In [18]:
#tokenizer.word_index

In [19]:
word_idx = tokenizer.word_index #Last is the key
idx_word = tokenizer.index_word

In [33]:
word_idx['the']

1

In [34]:
word_idx['project']

48

In [20]:
word_counts = tokenizer.word_counts
num_words = len(word_counts)
num_words

In [35]:
text_data[0][:200]

'The Project Gutenberg EBook of Stray Birds, by Rabindranath Tagore\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away o'

In [36]:
sequences = tokenizer.texts_to_sequences(text_data)
sequences

[[1,
  48,
  42,
  251,
  2,
  2295,
  749,
  34,
  511,
  454,
  16,
  251,
  8,
  15,
  1,
  170,
  2,
  591,
  1173,
  30,
  41,
  628,
  4,
  13,
  646,
  41,
  1357,
  1439,
  12,
  90,
  277,
  9,
  125,
  9,
  99,
  32,
  1309,
  170,
  9,
  152,
  1,
  155,
  2,
  1,
  48,
  42,
  218,
  984,
  13,
  16,
  251,
  32,
  670,
  30,
  408,
  42,
  281,
  1754,
  2295,
  749,
  1050,
  511,
  454,
  4979,
  772,
  1194,
  2854,
  8954,
  251,
  7652,
  1627,
  772,
  3162,
  4324,
  154,
  660,
  2503,
  1755,
  3478,
  547,
  459,
  1011,
  205,
  2600,
  1116,
  550,
  2,
  16,
  48,
  42,
  251,
  2295,
  749,
  699,
  34,
  3163,
  4325,
  2855,
  4,
  3479,
  3480,
  2295,
  749,
  34,
  511,
  454,
  1515,
  29,
  715,
  3,
  459,
  34,
  1,
  1050,
  221,
  2088,
  1,
  1558,
  1155,
  5430,
  4628,
  5,
  4326,
  34,
  6697,
  6698,
  3,
  10888,
  10889,
  2,
  7653,
  76,
  2295,
  749,
  2,
  1516,
  67,
  3,
  11,
  562,
  3,
  481,
  4,
  1239,
  99,
  4,
  2296,
  381

In [22]:
features = []
labels = []

# similar to lookbak in earlier files that we have used, here training_length = 50 is used

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

In [37]:
print(features[0]), print(labels[0])
print(features[1]), print(labels[1])

[2, 6, 515, 37, 1041, 5, 1, 980, 2, 6, 507, 16, 43, 5, 19, 8, 1169, 35, 2128]
25
[12176, 12177, 12178, 4, 1, 670, 623, 2433, 2434, 30, 942, 408, 2913, 678, 16, 1055, 17, 699, 29]
3252


(None, None)

In [23]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.75 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

In [38]:
!ls -la

total 4128
drwxr-xr-x   9 vidyadharbendre  staff     288 Apr 26 16:36 .
drwxr-xr-x  44 vidyadharbendre  staff    1408 Apr 26 16:25 ..
-rw-r--r--@  1 vidyadharbendre  staff    6148 Apr 26 16:25 .DS_Store
drwxr-xr-x@  8 vidyadharbendre  staff     256 Apr 26 14:40 .ipynb_checkpoints
-rw-r--r--@  1 vidyadharbendre  staff  467265 Apr 25 22:07 DNN_1.ipynb
-rw-r--r--@  1 vidyadharbendre  staff  713247 Apr 26 14:38 LSTM_3.ipynb
-rw-r--r--@  1 vidyadharbendre  staff  713513 Apr 25 22:07 RNN_2.ipynb
-rw-r--r--@  1 vidyadharbendre  staff  198645 Apr 26 16:36 Tagore_4.ipynb
drwxr-xr-x   3 vidyadharbendre  staff      96 Apr 26 16:27 images


In [ ]:
![Alt Text](./images/features_lables.png)

In [46]:
!ls -la ./

-rw-r--r--@ 1 vidyadharbendre  staff  482472 Apr 26 16:19 ./images/features_labels.png


In [55]:
![Features and Labels](features_labels.png)

zsh:1: bad pattern: [Features


![Features Vs Lables](images/features_labels.png)

In [ ]:
# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

In [24]:
for i, sequence in enumerate(X_train[:1]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: of a unity which appears in the aspect of a tree this one in me is creative its creations

Label: are



In [25]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=num_words,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         2459500   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 24595)             1598675   
                                                                 
Total params: 4104575 (15.66 MB)
Trainable params: 4104575 (15.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
h = model.fit(X_train, y_train, epochs = 1, batch_size = 100, verbose = 1)

45/45 [==============================] - 2s 27ms/step - loss: 9.4141 - accuracy: 0.0473


In [27]:
# save the model to file
model.save('../../Tagore/model_save/model_1000epochs.h5')

/Users/vidyadharbendre/miniforge3/envs/nndl_env/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
1
# load the model
model = load_model('../../Tagore/model_save/model_1000epochs.h5')

In [29]:
model.fit(X_train, y_train, batch_size=50, epochs=1000)

Epoch 1/1000
90/90 [==============================] - 2s 20ms/step - loss: 6.7959 - accuracy: 0.0560
Epoch 2/1000
90/90 [==============================] - 2s 20ms/step - loss: 6.4471 - accuracy: 0.0593
Epoch 3/1000
90/90 [==============================] - 2s 20ms/step - loss: 6.2476 - accuracy: 0.0593
Epoch 4/1000
90/90 [==============================] - 2s 20ms/step - loss: 6.1133 - accuracy: 0.0562
Epoch 5/1000
90/90 [==============================] - 2s 20ms/step - loss: 6.0328 - accuracy: 0.0633
Epoch 6/1000
90/90 [==============================] - 2s 20ms/step - loss: 5.9578 - accuracy: 0.0580
Epoch 7/1000
90/90 [==============================] - 2s 20ms/step - loss: 5.8917 - accuracy: 0.0571
Epoch 8/1000
90/90 [==============================] - 2s 20ms/step - loss: 5.8294 - accuracy: 0.0638
Epoch 9/1000
90/90 [==============================] - 2s 20ms/step - loss: 5.7639 - accuracy: 0.0636
Epoch 10/1000
90/90 [==============================] - 2s 21ms/step - loss: 5.6736 - accura

90/90 [==============================] - 2s 21ms/step - loss: 1.5328 - accuracy: 0.5989
Epoch 82/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.5179 - accuracy: 0.5998
Epoch 83/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.4891 - accuracy: 0.5987
Epoch 84/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.5054 - accuracy: 0.6018
Epoch 85/1000
90/90 [==============================] - 2s 22ms/step - loss: 1.4759 - accuracy: 0.5998
Epoch 86/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.4536 - accuracy: 0.6116
Epoch 87/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.4494 - accuracy: 0.6073
Epoch 88/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.4267 - accuracy: 0.6178
Epoch 89/1000
90/90 [==============================] - 2s 22ms/step - loss: 1.3955 - accuracy: 0.6273
Epoch 90/1000
90/90 [==============================] - 2s 21ms/step - loss: 1.3944 - accuracy: 0

90/90 [==============================] - 2s 21ms/step - loss: 0.7825 - accuracy: 0.7607
Epoch 161/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7809 - accuracy: 0.7613
Epoch 162/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7502 - accuracy: 0.7771
Epoch 163/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7334 - accuracy: 0.7742
Epoch 164/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7745 - accuracy: 0.7649
Epoch 165/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7533 - accuracy: 0.7616
Epoch 166/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7636 - accuracy: 0.7727
Epoch 167/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7481 - accuracy: 0.7796
Epoch 168/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7471 - accuracy: 0.7713
Epoch 169/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.7374 - ac

90/90 [==============================] - 2s 22ms/step - loss: 0.5580 - accuracy: 0.8196
Epoch 240/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5217 - accuracy: 0.8407
Epoch 241/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5497 - accuracy: 0.8273
Epoch 242/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5528 - accuracy: 0.8276
Epoch 243/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5535 - accuracy: 0.8238
Epoch 244/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.5293 - accuracy: 0.8307
Epoch 245/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5368 - accuracy: 0.8300
Epoch 246/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5555 - accuracy: 0.8173
Epoch 247/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5668 - accuracy: 0.8149
Epoch 248/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.5299 - ac

90/90 [==============================] - 2s 23ms/step - loss: 0.4252 - accuracy: 0.8664
Epoch 319/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.4356 - accuracy: 0.8600
Epoch 320/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.4315 - accuracy: 0.8671
Epoch 321/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4459 - accuracy: 0.8589
Epoch 322/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4110 - accuracy: 0.8664
Epoch 323/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4549 - accuracy: 0.8544
Epoch 324/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4325 - accuracy: 0.8524
Epoch 325/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4385 - accuracy: 0.8604
Epoch 326/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4272 - accuracy: 0.8633
Epoch 327/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.4078 - ac

...

90/90 [==============================] - 2s 23ms/step - loss: 0.2806 - accuracy: 0.9091
Epoch 635/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2706 - accuracy: 0.9151
Epoch 636/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2581 - accuracy: 0.9167
Epoch 637/1000
90/90 [==============================] - 2s 21ms/step - loss: 0.2816 - accuracy: 0.9080
Epoch 638/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2726 - accuracy: 0.9080
Epoch 639/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2750 - accuracy: 0.9056
Epoch 640/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.2914 - accuracy: 0.9058
Epoch 641/1000
90/90 [==============================] - 2s 22ms/step - loss: 0.2698 - accuracy: 0.9120
Epoch 642/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2825 - accuracy: 0.9044
Epoch 643/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2632 - ac

90/90 [==============================] - 2s 24ms/step - loss: 0.2443 - accuracy: 0.9198
Epoch 714/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2653 - accuracy: 0.9113
Epoch 715/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2622 - accuracy: 0.9140
Epoch 716/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2649 - accuracy: 0.9127
Epoch 717/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2696 - accuracy: 0.9096
Epoch 718/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2634 - accuracy: 0.9191
Epoch 719/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2641 - accuracy: 0.9149
Epoch 720/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2291 - accuracy: 0.9260
Epoch 721/1000
90/90 [==============================] - 2s 23ms/step - loss: 0.2636 - accuracy: 0.9120
Epoch 722/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2776 - ac

90/90 [==============================] - 2s 24ms/step - loss: 0.2354 - accuracy: 0.9227
Epoch 793/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2561 - accuracy: 0.9204
Epoch 794/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2495 - accuracy: 0.9158
Epoch 795/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2426 - accuracy: 0.9198
Epoch 796/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2377 - accuracy: 0.9231
Epoch 797/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2610 - accuracy: 0.9144
Epoch 798/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2345 - accuracy: 0.9222
Epoch 799/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2525 - accuracy: 0.9196
Epoch 800/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2670 - accuracy: 0.9164
Epoch 801/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2346 - ac

90/90 [==============================] - 2s 25ms/step - loss: 0.2220 - accuracy: 0.9258
Epoch 872/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2392 - accuracy: 0.9200
Epoch 873/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2429 - accuracy: 0.9191
Epoch 874/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2336 - accuracy: 0.9238
Epoch 875/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2364 - accuracy: 0.9233
Epoch 876/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2395 - accuracy: 0.9238
Epoch 877/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2404 - accuracy: 0.9198
Epoch 878/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2331 - accuracy: 0.9218
Epoch 879/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2259 - accuracy: 0.9222
Epoch 880/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2373 - ac

90/90 [==============================] - 2s 25ms/step - loss: 0.2405 - accuracy: 0.9247
Epoch 951/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2288 - accuracy: 0.9258
Epoch 952/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2379 - accuracy: 0.9233
Epoch 953/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2407 - accuracy: 0.9218
Epoch 954/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2206 - accuracy: 0.9258
Epoch 955/1000
90/90 [==============================] - 2s 24ms/step - loss: 0.2296 - accuracy: 0.9242
Epoch 956/1000
90/90 [==============================] - 2s 25ms/step - loss: 0.2415 - accuracy: 0.9193
Epoch 957/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2457 - accuracy: 0.9184
Epoch 958/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2189 - accuracy: 0.9340
Epoch 959/1000
90/90 [==============================] - 2s 26ms/step - loss: 0.2143 - ac

In [30]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

225/225 [==============================] - 1s 6ms/step - loss: 0.0124 - accuracy: 0.9953
[0.012443716637790203, 0.9953333139419556]

Model Performance: Log Loss and Accuracy on validation data
75/75 [==============================] - 0s 6ms/step - loss: 31.4420 - accuracy: 0.3800
[31.44198226928711, 0.3799999952316284]


In [31]:
# Generate Text

In [32]:
seed_length=20
new_words=20
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

  
        # Softmax
        preds = preds / sum(preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

1/1 [==============================] - 0s 10ms/step
Original Sequence: 
gutenberg tm license terms from this work or any files containing a part of this work or any other work


Generated Sequence: 
i on all over and the file was first posted on june 4 ebooks were prepared by thousands of volunteers
